In [25]:
import os
from pprint import pprint

from dotenv import find_dotenv, load_dotenv
from qdrant_client import QdrantClient
from qdrant_client.http.models import Distance, VectorParams

load_dotenv(find_dotenv())

True

Connect to your Qdrant cluster

In [20]:
qdrant_client = QdrantClient(
    url=os.environ.get("QDRANT_CLUSTER_URL"),
    api_key=os.environ.get("QDRANT_API_KEY"),
)

Create collection

In [21]:
def create_collection(name: str, vector_size: int = 1536) -> bool:
    return qdrant_client.recreate_collection(
        collection_name=name,
        vectors_config=VectorParams(size=vector_size, distance=Distance.COSINE),
    )

In [22]:
create_collection("test_collection")

True

Delete collection

In [6]:
def delete_collection(collection: str, timeout: int = None) -> bool:
    return qdrant_client.delete_collection(collection_name=collection, timeout=timeout)

In [15]:
delete_collection(collection="test_collection")

True

In [18]:
def get_collection_info(collection: str) -> dict:
    return qdrant_client.get_collection(collection_name=collection).model_dump()

In [24]:
response = get_collection_info(collection="test_collection")

In [26]:
pprint(response)

{'config': {'hnsw_config': {'ef_construct': 100,
                            'full_scan_threshold': 10000,
                            'm': 16,
                            'max_indexing_threads': 0,
                            'on_disk': False,
                            'payload_m': None},
            'optimizer_config': {'default_segment_number': 0,
                                 'deleted_threshold': 0.2,
                                 'flush_interval_sec': 5,
                                 'indexing_threshold': 20000,
                                 'max_optimization_threads': None,
                                 'max_segment_size': None,
                                 'memmap_threshold': None,
                                 'vacuum_min_vector_number': 1000},
            'params': {'on_disk_payload': True,
                       'read_fan_out_factor': None,
                       'replication_factor': 1,
                       'shard_number': 1,
                       

In [39]:
import json
import tiktoken

In [46]:
encoder = tiktoken.encoding_for_model(model_name="gpt-4-1106-preview")

In [ ]:
encoder.encode()

In [29]:
raw_data_path = "../scraper/srb_labor_law_data.json"

In [31]:
with open(raw_data_path, "r", encoding="utf-8") as file:
    data = json.loads(file.read())

In [48]:
for sample in data:
    title = sample["title"]
    texts = " ".join(sample["texts"])
    print(". ".join([title, texts]))
    print(len(encoder.encode(". ".join([title, texts]))))

Član 1. Prava, obaveze i odgovornosti iz radnog odnosa, odnosno po osnovu rada, uređuju se ovim zakonom i posebnim zakonom, u skladu sa ratifikovanim međunarodnim konvencijama. Prava, obaveze i odgovornosti iz radnog odnosa uređuju se i kolektivnim ugovorom i ugovorom o radu, a pravilnikom o radu, odnosno ugovorom o radu - samo kada je to ovim zakonom određeno.
145
Član 2. Odredbe ovog zakona primenjuju se na zaposlene koji rade na teritoriji Republike Srbije, kod domaćeg ili stranog pravnog, odnosno fizičkog lica (u daljem tekstu: poslodavac), kao i na zaposlene koji su upućeni na rad u inostranstvo od strane poslodavca ako zakonom nije drukčije određeno. Odredbe ovog zakona primenjuju se i na zaposlene u državnim organima, organima teritorijalne autonomije i lokalne samouprave i javnim službama, ako zakonom nije drukčije određeno. Odredbe ovog zakona primenjuju se i na zaposlene kod poslodavaca u oblasti saobraćaja, ako posebnim propisom nije drukčije određeno. Odredbe ovog zakona pr